# 🤗 Aplicaciones prácticas de Transformers con Hugging Face

En esta clase vamos a usar modelos preentrenados en español para resolver tareas reales de Procesamiento del Lenguaje Natural (PLN), sin necesidad de entrenar modelos desde cero.

Trabajaremos con la librería `transformers` de Hugging Face, que permite usar modelos de tipo BERT, GPT y similares en muy pocas líneas de código.

---


In [45]:
# Instalación de Hugging Face Transformers (solo una vez)
!pip install -q transformers

## 1. Cargando el pipeline de Hugging Face

Hugging Face proporciona "pipelines" que encapsulan todo el proceso: tokenización, modelo y decodificación. Solo tenés que indicar qué tarea querés hacer.


In [46]:
from transformers import pipeline

## 2. Análisis de sentimiento en español

Vamos a usar un modelo entrenado para identificar si una frase expresa un **sentimiento positivo o negativo**. Este modelo fue entrenado con tweets en español.

In [47]:
sentiment = pipeline("sentiment-analysis", model="finiteautomata/beto-sentiment-analysis")

Device set to use cuda:0


In [48]:
frases = [
    "La verdad, este lugar está increible! Me encantó!",
    "Una porquería de servicio, nunca más vengo",
    "Me encantan las mesas, lastima el baño que es un desastre",
    "Me enviaron un producto llegó dañado. Qué bajon",
    "Todo impecable. De primera",
    "Qué afano, me cuentearon con el producto",
    "Muy conforme con el resultado final",
    "No me gustó para nada la experiencia",
    "Superó mis expectativas, ¡gracias!",
    "No lo recomiendo, mala calidad",
    "Me vino todo roto, exijo una devolucion",
    "Muy buen servicio, lo recomiendo",
    "Todo impecable, llego en tiempo y forma"
]

In [49]:
for frase in frases:
    print(f"{frase} → {sentiment(frase)}")

La verdad, este lugar está increible! Me encantó! → [{'label': 'POS', 'score': 0.998566210269928}]
Una porquería de servicio, nunca más vengo → [{'label': 'NEG', 'score': 0.9994451403617859}]
Me encantan las mesas, lastima el baño que es un desastre → [{'label': 'NEG', 'score': 0.9644700884819031}]
Me enviaron un producto llegó dañado. Qué bajon → [{'label': 'NEG', 'score': 0.9993813037872314}]
Todo impecable. De primera → [{'label': 'POS', 'score': 0.9977364540100098}]
Qué afano, me cuentearon con el producto → [{'label': 'NEU', 'score': 0.544805645942688}]
Muy conforme con el resultado final → [{'label': 'POS', 'score': 0.9939767122268677}]
No me gustó para nada la experiencia → [{'label': 'NEG', 'score': 0.9992014765739441}]
Superó mis expectativas, ¡gracias! → [{'label': 'POS', 'score': 0.9984914064407349}]
No lo recomiendo, mala calidad → [{'label': 'NEG', 'score': 0.9143223762512207}]
Me vino todo roto, exijo una devolucion → [{'label': 'NEG', 'score': 0.9983291029930115}]
Muy bu

## 3. Clasificación de texto por tema (*zero-shot*)

¿Querés clasificar frases por categorías sin entrenar un modelo? ¡Esto es posible gracias al aprendizaje **zero-shot**!

El modelo puede asociar un texto con una o más **etiquetas** sugeridas por vos, aunque nunca fue entrenado específicamente para esas clases.

In [50]:
classifier = pipeline("zero-shot-classification", model="Recognai/bert-base-spanish-wwm-cased-xnli")

Device set to use cuda:0


In [51]:
texto = "Tralalero tralala se hizo famoso por ser una rima particular de italia que trascendio por todo el mundo"

etiquetas = ["animales", "personajes", "niños", "espectáculos"]

print(classifier(texto, candidate_labels=etiquetas))

{'sequence': 'Tralalero tralala se hizo famoso por ser una rima particular de italia que trascendio por todo el mundo', 'labels': ['personajes', 'espectáculos', 'niños', 'animales'], 'scores': [0.7077112197875977, 0.2328801304101944, 0.031039604917168617, 0.02836904115974903]}


## 4. Resumen automático de textos

Este pipeline toma un texto largo y genera un resumen breve en español. Ideal para noticias, informes o textos descriptivos.

Usamos un modelo BERT2BERT adaptado al español.

In [52]:
summarizer = pipeline(
    "summarization",
    model="csebuetnlp/mT5_multilingual_XLSum",
    tokenizer="csebuetnlp/mT5_multilingual_XLSum"
)

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Device set to use cuda:0


In [53]:
parrafo = """
El Ministerio de Salud confirmó hoy que se ha logrado una reducción sostenida de casos de dengue en las últimas semanas.
Las campañas de prevención, sumadas a la llegada del frío, habrían contribuido a esta baja. Sin embargo, se pide a la población mantener las precauciones.
"""

resumen = summarizer(parrafo, max_length=50, min_length=20, do_sample=False)
print(resumen[0]['summary_text'])


El número de casos de dengue en Estados Unidos alcanzó un nuevo récord.


In [54]:
texto = """
La inflación en Argentina ha mostrado una leve desaceleración en el último mes, según el informe del INDEC.
Sin embargo, los analistas advierten que la tendencia aún no se revierte, y que podrían esperarse aumentos para el próximo trimestre.
"""

resumen = summarizer(texto, max_length=60, min_length=25, do_sample=False)
print(resumen[0]['summary_text'])


El Fondo Monetario Internacional (INDEC) dijo que la inflación en Argentina se ha recuperado de una desaceleración en el último mes.


## 5. Traducción automática (Español → Inglés)

También podemos usar modelos preentrenados para **traducir textos**. En este caso, usaremos uno especializado para traducir del español al inglés.


In [55]:
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cuda:0


In [56]:
texto = "La familia es lo mas importante en este mundo."

print(translator(texto)[0]['translation_text'])


Family is the most important thing in this world.


## 6. Generación de texto en español (GPT)

Con un modelo tipo GPT entrenado en español, podemos **generar texto a partir de un inicio dado**. Ideal para escribir contenido creativo, continuar frases, etc.


In [57]:
generator = pipeline("text-generation", model="PlanTL-GOB-ES/gpt2-base-bne")

Device set to use cuda:0


In [58]:
prompt = "Un dinosaurio es"

resultado = generator(prompt, max_length=100, num_return_sequences=1)
print(resultado[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=256) and `max_length`(=101) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Un dinosaurio es un ser vivo que está en el medio rural, pero es un dinosaurio. 


## 7. Reflexión y discusión

- ¿Cuál de estos pipelines te pareció más sorprendente o útil?
- ¿Creés que estas herramientas podrían usarse en un proyecto real? ¿En cuál?
- ¿Notaste errores o sesgos? ¿Por qué creés que aparecen?



## 8. Actividad libre (opcional si hay tiempo)

Explorá uno de los pipelines y diseñá tu propio experimento:

- Probá frases con sarcasmo o jergas locales.
- Resumí un artículo de Wikipedia.
- Traducí algo complejo (tecnológico, poético, etc.).
- Completá una frase usando estilo formal o informal.

Al final compartimos los hallazgos más interesantes con el grupo 👀



a) Probando frases random

In [59]:
frase_practica = [
    "Si te paras de manos, te paro el carro",
    "Estan mansas las semitas niño, mas buenas que la no se que",
    "Quien sos vo, que te haces el mas importante",
    "Estas mas linda que una patada en la nuca",
    "Hablando de Roma, llego Roman",
    "Que talca? quien pudiera irse temprano a la choza",
    "Me pedi una milanga que estaban como ojota en el desierto"
]

for frase in frase_practica:
    print(f"{frase} → {sentiment(frase)}")

Si te paras de manos, te paro el carro → [{'label': 'NEU', 'score': 0.9961426854133606}]
Estan mansas las semitas niño, mas buenas que la no se que → [{'label': 'POS', 'score': 0.9983853101730347}]
Quien sos vo, que te haces el mas importante → [{'label': 'POS', 'score': 0.9013652801513672}]
Estas mas linda que una patada en la nuca → [{'label': 'POS', 'score': 0.9959981441497803}]
Hablando de Roma, llego Roman → [{'label': 'NEU', 'score': 0.9949650168418884}]
Que talca? quien pudiera irse temprano a la choza → [{'label': 'NEU', 'score': 0.9930123686790466}]
Me pedi una milanga que estaban como ojota en el desierto → [{'label': 'NEU', 'score': 0.9813180565834045}]


b) Resumir una pagina de wikipedia

In [60]:
texto_wiki = "El hornero común (Furnarius rufus) es una especie de ave paseriforme de la familia de los furnáridos endémica de América del Sur. Es un pájaro pequeño que mide entre 16 y 23 cm de longitud y no presenta dimorfismo sexual aparente. Es un ave insectívora que consigue su alimento mientras camina por el suelo. Su dieta consiste principalmente en pequeños invertebrados como coleópteros, grillos, mariposas y otros insectos. Se trata de una especie monógina que construye un característico nido de barro en árboles, construcciones y otras estructuras. La hembra deposita de tres a cuatro huevos, que ambos sexos ayudan a incubar. Al ser un ave de hábitos no migratorios, vive y se reproduce en su área de residencia. Debido a su amplia área de distribución y su abundancia en incremento, el hornero común es clasificado como de preocupación menor por la Unión Internacional para la Conservación de la Naturaleza. Esta ave se favoreció con la presencia del ser humano, convirtiéndose en el centro de muchas leyendas y canciones pertenecientes al folclore de América del Sur. Además, los agricultores admiten al hornero común ya que este protege el sembrado de insectos dañinos. Es el ave nacional de Argentina.[4]​ Apareció en la moneda argentina de medio centavo de austral, acuñada en 1985, y a partir de 2017 en el billete de mil pesos."

resumen = summarizer(texto_wiki, max_length=100, min_length=50, do_sample=False)
print(resumen[0]['summary_text'])

El hornero común (Furnarius rufus) es un pájaro paseriforme de la familia de los furnáridos endémica de América del Sur, que vive y se reproduce en su área de residencia.


c) Traduccion

In [61]:
Transcript = "Durante el entrenamiento, la red neuronal utiliza un bucle de retroalimentación que funciona de la siguiente manera: Cada nodo intenta adivinar el siguiente nodo de la ruta. Se comprueba si la suposición es correcta. Los nodos asignan valores de peso más altos a las rutas que conducen a más suposiciones correctas y valores de peso más bajos a las rutas de los nodos que conducen a suposiciones incorrectas."
print(translator(Transcript)[0]['translation_text'])

During training, the neural network uses a feedback loop that works as follows: Each node tries to guess the next node of the path. It checks if the assumption is correct. The nodes assign higher weight values to the routes that lead to more correct assumptions and lower weight values to the routes of the nodes that lead to incorrect assumptions.


d) Completar una frase

In [63]:
prompt_2 = "El amor es un sentimiento"

resultado = generator(prompt_2, max_length=100, num_return_sequences=1)
print(resultado[0]['generated_text'])

Both `max_new_tokens` (=256) and `max_length`(=101) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


El amor es un sentimiento que merece ser respetado, que no puede ser compartido, pero que es un sentimiento de amor que se debe hacer presente, que debe estar en el mundo de los ciudadanos, de los ciudadanos, de los ciudadanos... 
